# How much average monthly income I could expect from my investments in terms of lease/rent after upgrading properties, paying mortgage, and paying taxes. Calculate my ~ROI.

In [48]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import us

ModuleNotFoundError: No module named 'us'

## Assumptions

- Apply an average upgrade cost to all properties (2%-6%, assume 4%) [Google](https://www.google.com/search?sca_esv=f9439907e5543505&rlz=1C5CHFA_enUS1066US1066&sxsrf=ADLYWIKGALe9lErWdMYpzzi_ACDXvpSxuQ:1733020033405&q=So+if+I+buy+a+house,+what+is+the+average+cost+to+spruce+up+the+house+(i.+paint,+carpet,+etc)?+Is+there+a+percentage+I+can+apply+to+the+home+value+to+get+this+value?&spell=1&sa=X&ved=2ahUKEwjY-OrbwoWKAxVarYkEHXoMMNsQBSgAegQIDRAB&biw=2560&bih=1318&dpr=1)
- Mortage rate: 6.31% **Source:** [Bankrate](https://www.bankrate.com/mortgages/mortgage-rates/)
- 30 year fixed mortgage, 20% down payment
- Property tax rates by state imported as property_tax_rates.csv **Source:** [Maptive](https://www.maptive.com/interactive-maps/property-taxes-by-state/)
- Renters pay all utilities
- Rent rates: Imported as rent_rates.csv **Source:** [Rentdata](https://www.rentdata.org/states/2024)

## Rent analysis for 1-4 bedroom properties (Single Family and Condo included)
- We are not analyzing 5+ bedroom data from Zillow since we can't determine the rent per bedroom since all properties with 5+ bedrooms are included.

### Data Loading

In [ ]:
# File paths
tax_rates_file_path = 'property_tax_rates.csv'
rent_rates_file_path = 'rent_rates.csv'
price_data_1br_path = 'Zip_zhvi_bdrmcnt_1_uc_sfrcondo_tier_0.33_0.67_sm_sa_month.csv'
price_data_2br_path = 'Zip_zhvi_bdrmcnt_2_uc_sfrcondo_tier_0.33_0.67_sm_sa_month.csv'
price_data_3br_path = 'Zip_zhvi_bdrmcnt_3_uc_sfrcondo_tier_0.33_0.67_sm_sa_month.csv'
price_data_4br_path = 'Zip_zhvi_bdrmcnt_4_uc_sfrcondo_tier_0.33_0.67_sm_sa_month.csv'

# Load the CSV files into pandas DataFrames
tax_rates = pd.read_csv(tax_rates_file_path)
rent_rates = pd.read_csv(rent_rates_file_path)
price_data_1br = pd.read_csv(price_data_1br_path)
price_data_2br = pd.read_csv(price_data_2br_path)
price_data_3br = pd.read_csv(price_data_3br_path)
price_data_4br = pd.read_csv(price_data_4br_path)

# State names in the property_tax_rates and rent_rates
# need to be converted into a abbreviations to easily join data from the Zillow data set
# Use the us Python library to do this easily
tax_rates['State'] = tax_rates['State'].apply(lambda x: us.states.lookup(x).abbr)
rent_rates['State'] = rent_rates['State'].apply(lambda x: us.states.lookup(x).abbr)

### Data Cleaning

In [ ]:
# Since we're analyze the most recent house price data (October 2024), we can ignore all other months if they are blank.
# However we should remove rows where October 2024 data is missing from all price data

# Using 10/31/24 as the column key was giving an error
oct_2024_column = '2024-10-31'

# Keep rows with October 2024 data in the cell since we're only analyzing that month
price_data_1br = price_data_1br[price_data_1br[oct_2024_column].notna()]
price_data_2br = price_data_2br[price_data_2br[oct_2024_column].notna()]
price_data_3br = price_data_3br[price_data_3br[oct_2024_column].notna()]
price_data_4br = price_data_4br[price_data_4br[oct_2024_column].notna()]

### Calculations assuming buying a house with a 20% downpayment mortgage

In [ ]:
# Constants for calculation (taken from Assumptions section)
repair_cost_rate = 0.04
down_payment_rate = 0.20
mortgage_rate = 0.0631

# Filter tax rates by state and year (2024)
tax_rates_2024 = tax_rates[['State', '2024']]

# Function to calculate NOI for each row
def calculate_noi(row, bedroom_size):
    price = row[oct_2024_column]
    display(price)
    state = row['State']
    display(state)
    
    # Get tax rate
    tax_rate = tax_rates_2024.loc[tax_rates_2024['State'] == state, '2024'].values
    display(tax_rate)
    
    # Get rent rate
    rent_rate = rent_rates.loc[rent_rates['State'] == state, bedroom_size].values
    
    tax_rate = tax_rate[0]
    rent_rate = rent_rate[0]
    
    # Calculate expenses
    annual_taxes = price * tax_rate
    # annual_repairs = price * repair_cost_rate
    loan_amount = price * (1 - down_payment_rate)
    
    # Mortgage payment calculation
    monthly_mortgage_payment = loan_amount * (monthly_interest_rate * (1 + monthly_interest_rate)**total_payments) / ((1 + monthly_interest_rate)**total_payments - 1)
    annual_mortgage_payment = monthly_mortgage_payment * 12
    
    annual_rent_income = rent_rate * 12
    
    # NOI calculation
    noi = annual_rent_income - (annual_taxes + annual_mortgage_payment)
    return noi

# Apply the function to calculate Net Operating Income (NOI) for each dataset
price_data_1br['NOI'] = price_data_1br.apply(lambda row: calculate_noi(row, '1 BR'), axis=1)
price_data_2br['NOI'] = price_data_2br.apply(lambda row: calculate_noi(row, '2 BR'), axis=1)
price_data_3br['NOI'] = price_data_3br.apply(lambda row: calculate_noi(row, '3 BR'), axis=1)
price_data_4br['NOI'] = price_data_4br.apply(lambda row: calculate_noi(row, '4 BR'), axis=1)

# Display the first few rows with NOI for each dataset
print("1 Bedroom NOI Data:")
print(price_data_1br['NOI'].nlargest(3))

print("\n2 Bedroom NOI Data:")
print(price_data_2br['NOI'].nlargest(3))

print("\n3 Bedroom NOI Data:")
print(price_data_3br['NOI'].nlargest(3))

print("\n4 Bedroom NOI Data:")
print(price_data_4br['NOI'].nlargest(3))